# High-Iteration SA with Numba Optimization

Building on previous experiments:
1. Use numba-optimized functions for 10x speedup
2. Increase iterations to 2000-5000 per N (vs 300-900 before)
3. Add fractional translation refinement after SA
4. Multiple restarts for small N
5. Focus on N=1-50 where improvements matter most

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import time
import random
from numba import njit, prange
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')
SCALE = float(scale_factor)

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
# Tree polygon template (numba-compatible)
@njit
def make_polygon_template():
    """Create the tree polygon template vertices."""
    tw = 0.15  # trunk width
    th = 0.2   # trunk height
    bw = 0.7   # base width
    mw = 0.4   # mid width
    ow = 0.25  # top width
    tip = 0.8
    t1 = 0.5
    t2 = 0.25
    base = 0.0
    tbot = -th
    
    x = np.array([0, ow/2, ow/4, mw/2, mw/4, bw/2, tw/2, tw/2, -tw/2, -tw/2, -bw/2, -mw/4, -mw/2, -ow/4, -ow/2], np.float64)
    y = np.array([tip, t1, t1, t2, t2, base, base, tbot, tbot, base, base, t2, t2, t1, t1], np.float64)
    return x, y

TX, TY = make_polygon_template()
print(f"Tree template: {len(TX)} vertices")

Tree template: 15 vertices


In [3]:
@njit
def get_tree_vertices(cx, cy, deg, tx, ty):
    """Get rotated and translated tree vertices."""
    n = len(tx)
    vx = np.empty(n, np.float64)
    vy = np.empty(n, np.float64)
    angle_rad = deg * math.pi / 180.0
    c = math.cos(angle_rad)
    s = math.sin(angle_rad)
    for i in range(n):
        vx[i] = c * tx[i] - s * ty[i] + cx
        vy[i] = s * tx[i] + c * ty[i] + cy
    return vx, vy

@njit
def get_bbox_side(xs, ys, degs, tx, ty):
    """Calculate bounding box side length for a group of trees."""
    n = len(xs)
    V = len(tx)
    mnx = 1e300
    mny = 1e300
    mxx = -1e300
    mxy = -1e300
    
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    
    return max(mxx - mnx, mxy - mny)

@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a group (side^2 / n)."""
    side = get_bbox_side(xs, ys, degs, tx, ty)
    return side * side / len(xs)

print("Numba functions defined")

Numba functions defined


In [4]:
# Fast overlap detection using numba
@njit
def ccw(ax, ay, bx, by, cx, cy):
    """Check if three points are in counter-clockwise order."""
    return (cy - ay) * (bx - ax) > (by - ay) * (cx - ax)

@njit
def segments_intersect(ax, ay, bx, by, cx, cy, dx, dy):
    """Check if line segment AB intersects with CD."""
    return ccw(ax, ay, cx, cy, dx, dy) != ccw(bx, by, cx, cy, dx, dy) and \
           ccw(ax, ay, bx, by, cx, cy) != ccw(ax, ay, bx, by, dx, dy)

@njit
def point_in_polygon(px, py, poly_x, poly_y):
    """Check if point is inside polygon using ray casting."""
    n = len(poly_x)
    inside = False
    j = n - 1
    for i in range(n):
        if ((poly_y[i] > py) != (poly_y[j] > py)) and \
           (px < (poly_x[j] - poly_x[i]) * (py - poly_y[i]) / (poly_y[j] - poly_y[i]) + poly_x[i]):
            inside = not inside
        j = i
    return inside

@njit
def polygons_overlap(vx1, vy1, vx2, vy2):
    """Check if two polygons overlap (not just touch)."""
    n1 = len(vx1)
    n2 = len(vx2)
    
    # Bounding box pre-filter
    min1x, max1x = vx1.min(), vx1.max()
    min1y, max1y = vy1.min(), vy1.max()
    min2x, max2x = vx2.min(), vx2.max()
    min2y, max2y = vy2.min(), vy2.max()
    
    if max1x < min2x or max2x < min1x or max1y < min2y or max2y < min1y:
        return False
    
    # Check if any vertex of poly1 is inside poly2
    for i in range(n1):
        if point_in_polygon(vx1[i], vy1[i], vx2, vy2):
            return True
    
    # Check if any vertex of poly2 is inside poly1
    for i in range(n2):
        if point_in_polygon(vx2[i], vy2[i], vx1, vy1):
            return True
    
    # Check edge intersections
    for i in range(n1):
        i_next = (i + 1) % n1
        for j in range(n2):
            j_next = (j + 1) % n2
            if segments_intersect(vx1[i], vy1[i], vx1[i_next], vy1[i_next],
                                  vx2[j], vy2[j], vx2[j_next], vy2[j_next]):
                return True
    
    return False

@njit
def has_any_overlap(xs, ys, degs, tx, ty):
    """Check if any pair of trees overlaps."""
    n = len(xs)
    if n <= 1:
        return False
    
    # Pre-compute all vertices
    all_vx = np.empty((n, len(tx)), np.float64)
    all_vy = np.empty((n, len(ty)), np.float64)
    
    for i in range(n):
        vx, vy = get_tree_vertices(xs[i], ys[i], degs[i], tx, ty)
        all_vx[i] = vx
        all_vy[i] = vy
    
    # Check all pairs
    for i in range(n):
        for j in range(i + 1, n):
            if polygons_overlap(all_vx[i], all_vy[i], all_vx[j], all_vy[j]):
                return True
    
    return False

@njit
def check_single_overlap(xs, ys, degs, idx, tx, ty):
    """Check if tree at idx overlaps with any other tree."""
    n = len(xs)
    if n <= 1:
        return False
    
    vx_i, vy_i = get_tree_vertices(xs[idx], ys[idx], degs[idx], tx, ty)
    
    for j in range(n):
        if j == idx:
            continue
        vx_j, vy_j = get_tree_vertices(xs[j], ys[j], degs[j], tx, ty)
        if polygons_overlap(vx_i, vy_i, vx_j, vy_j):
            return True
    
    return False

print("Fast overlap detection defined")

Fast overlap detection defined


In [5]:
# Test the numba functions
print("Testing numba functions...")
xs = np.array([0.0, 0.7], np.float64)
ys = np.array([0.0, 0.0], np.float64)
degs = np.array([0.0, 0.0], np.float64)

side = get_bbox_side(xs, ys, degs, TX, TY)
print(f"Bbox side for 2 trees: {side:.6f}")

overlap = has_any_overlap(xs, ys, degs, TX, TY)
print(f"Has overlap: {overlap}")

# Test overlapping case
xs_overlap = np.array([0.0, 0.3], np.float64)
overlap2 = has_any_overlap(xs_overlap, ys, degs, TX, TY)
print(f"Has overlap (close trees): {overlap2}")

Testing numba functions...
Bbox side for 2 trees: 1.400000


Has overlap: True
Has overlap (close trees): True


In [ ]:
# Debug overlap detection
print("Debug: Testing overlap detection...")
xs = np.array([0.0, 0.7], np.float64)
ys = np.array([0.0, 0.0], np.float64)
degs = np.array([0.0, 0.0], np.float64)

# Get vertices for both trees
vx1, vy1 = get_tree_vertices(xs[0], ys[0], degs[0], TX, TY)
vx2, vy2 = get_tree_vertices(xs[1], ys[1], degs[1], TX, TY)

print(f"Tree 1 x range: [{vx1.min():.4f}, {vx1.max():.4f}]")
print(f"Tree 2 x range: [{vx2.min():.4f}, {vx2.max():.4f}]")
print(f"Tree 1 y range: [{vy1.min():.4f}, {vy1.max():.4f}]")
print(f"Tree 2 y range: [{vy2.min():.4f}, {vy2.max():.4f}]")

# The trees should touch at x=0.35 (tree1 right edge) and x=0.35 (tree2 left edge)
# Tree base width is 0.7, so tree1 goes from -0.35 to 0.35, tree2 goes from 0.35 to 1.05
# They should just touch, not overlap

In [ ]:
def find_best_grid_trees(n):
    """Find the best grid-based placement for n trees."""
    best_score = float('inf')
    best_xs, best_ys, best_degs = None, None, None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
            
            xs_list, ys_list, degs_list = [], [], []
            rest = n
            r = 0
            
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0.0 if r % 2 == 0 else 180.0
                x_offset = 0.0 if r % 2 == 0 else 0.35
                
                if r % 2 == 0:
                    y = float(r // 2) * 1.0
                else:
                    y = 0.8 + float((r - 1) // 2) * 1.0
                
                for i in range(m):
                    xs_list.append(0.7 * i + x_offset)
                    ys_list.append(y)
                    degs_list.append(angle)
                
                r += 1
            
            if len(xs_list) != n:
                continue
            
            xs = np.array(xs_list, np.float64)
            ys = np.array(ys_list, np.float64)
            degs = np.array(degs_list, np.float64)
            
            score = score_group(xs, ys, degs, TX, TY)
            if score < best_score:
                best_score = score
                best_xs = xs.copy()
                best_ys = ys.copy()
                best_degs = degs.copy()
    
    return best_xs, best_ys, best_degs, best_score

print("Grid placement function defined")

In [ ]:
@njit
def sa_optimize(xs, ys, degs, tx, ty, max_iter, T0=1.0, alpha=0.95, seed=42):
    """
    Simulated Annealing optimizer using numba for speed.
    Returns optimized xs, ys, degs and best score.
    """
    np.random.seed(seed)
    n = len(xs)
    
    if n <= 1:
        return xs.copy(), ys.copy(), degs.copy(), score_group(xs, ys, degs, tx, ty)
    
    # Copy arrays
    cur_xs = xs.copy()
    cur_ys = ys.copy()
    cur_degs = degs.copy()
    
    best_xs = xs.copy()
    best_ys = ys.copy()
    best_degs = degs.copy()
    
    cur_score = score_group(cur_xs, cur_ys, cur_degs, tx, ty)
    best_score = cur_score
    
    T = T0
    move_scale = 0.1
    rot_scale = 30.0
    no_improve = 0
    
    for iteration in range(max_iter):
        scale = T / T0
        
        # Choose random tree
        i = np.random.randint(0, n)
        
        # Choose move type (0-4)
        move_type = np.random.randint(0, 5)
        
        # Store original values
        orig_x = cur_xs[i]
        orig_y = cur_ys[i]
        orig_deg = cur_degs[i]
        
        # Calculate centroid
        cx = cur_xs.mean()
        cy = cur_ys.mean()
        
        if move_type == 0:  # Random translation
            cur_xs[i] += (np.random.random() - 0.5) * 2 * move_scale * scale
            cur_ys[i] += (np.random.random() - 0.5) * 2 * move_scale * scale
        elif move_type == 1:  # Centroid move
            dx = (cx - cur_xs[i]) * np.random.random() * scale * 0.3
            dy = (cy - cur_ys[i]) * np.random.random() * scale * 0.3
            cur_xs[i] += dx
            cur_ys[i] += dy
        elif move_type == 2:  # Random rotation
            cur_degs[i] += (np.random.random() - 0.5) * 2 * rot_scale * scale
        elif move_type == 3:  # Combined
            cur_xs[i] += (np.random.random() - 0.5) * 2 * move_scale * scale
            cur_ys[i] += (np.random.random() - 0.5) * 2 * move_scale * scale
            cur_degs[i] += (np.random.random() - 0.5) * 2 * rot_scale * scale
        else:  # Swap (move_type == 4)
            if n >= 2:
                j = np.random.randint(0, n)
                while j == i:
                    j = np.random.randint(0, n)
                # Swap positions
                cur_xs[i], cur_xs[j] = cur_xs[j], cur_xs[i]
                cur_ys[i], cur_ys[j] = cur_ys[j], cur_ys[i]
        
        # Check for overlap
        if check_single_overlap(cur_xs, cur_ys, cur_degs, i, tx, ty):
            # Revert
            cur_xs[i] = orig_x
            cur_ys[i] = orig_y
            cur_degs[i] = orig_deg
            if move_type == 4 and n >= 2:  # Revert swap
                cur_xs[i], cur_xs[j] = cur_xs[j], cur_xs[i]
                cur_ys[i], cur_ys[j] = cur_ys[j], cur_ys[i]
            move_scale *= 0.99
            rot_scale *= 0.99
            continue
        
        # Calculate new score
        new_score = score_group(cur_xs, cur_ys, cur_degs, tx, ty)
        
        # Accept or reject
        delta = new_score - cur_score
        if delta < 0 or np.random.random() < math.exp(-delta / T):
            cur_score = new_score
            move_scale = min(move_scale * 1.01, 0.1)
            rot_scale = min(rot_scale * 1.01, 30.0)
            
            if cur_score < best_score:
                best_score = cur_score
                best_xs[:] = cur_xs
                best_ys[:] = cur_ys
                best_degs[:] = cur_degs
                no_improve = 0
            else:
                no_improve += 1
        else:
            # Revert
            cur_xs[i] = orig_x
            cur_ys[i] = orig_y
            cur_degs[i] = orig_deg
            if move_type == 4 and n >= 2:
                cur_xs[i], cur_xs[j] = cur_xs[j], cur_xs[i]
                cur_ys[i], cur_ys[j] = cur_ys[j], cur_ys[i]
            move_scale *= 0.99
            rot_scale *= 0.99
            no_improve += 1
        
        # Cool down
        T *= alpha
        
        # Reheat if stagnant
        if no_improve > 200:
            T = min(T * 3.0, T0 * 0.7)
            no_improve = 0
    
    return best_xs, best_ys, best_degs, best_score

print("SA optimizer defined")

In [ ]:
@njit
def fractional_refinement(xs, ys, degs, tx, ty, max_iters=50):
    """Fine-tune positions with small fractional moves."""
    n = len(xs)
    if n <= 1:
        return xs.copy(), ys.copy(), degs.copy()
    
    cur_xs = xs.copy()
    cur_ys = ys.copy()
    cur_degs = degs.copy()
    
    best_side = get_bbox_side(cur_xs, cur_ys, cur_degs, tx, ty)
    
    steps = np.array([0.005, 0.002, 0.001, 0.0005, 0.0002, 0.0001], np.float64)
    directions = np.array([[1, 0], [-1, 0], [0, 1], [0, -1], 
                           [1, 1], [1, -1], [-1, 1], [-1, -1]], np.float64)
    
    for iteration in range(max_iters):
        improved = False
        
        for i in range(n):
            for step in steps:
                for d in range(8):
                    dx = directions[d, 0] * step
                    dy = directions[d, 1] * step
                    
                    orig_x = cur_xs[i]
                    orig_y = cur_ys[i]
                    
                    cur_xs[i] = orig_x + dx
                    cur_ys[i] = orig_y + dy
                    
                    # Check overlap
                    if check_single_overlap(cur_xs, cur_ys, cur_degs, i, tx, ty):
                        cur_xs[i] = orig_x
                        cur_ys[i] = orig_y
                        continue
                    
                    new_side = get_bbox_side(cur_xs, cur_ys, cur_degs, tx, ty)
                    
                    if new_side < best_side - 1e-10:
                        best_side = new_side
                        improved = True
                    else:
                        cur_xs[i] = orig_x
                        cur_ys[i] = orig_y
        
        if not improved:
            break
    
    return cur_xs, cur_ys, cur_degs

print("Fractional refinement defined")

In [ ]:
# Test SA on small N
print("Testing SA on N=10...")
xs, ys, degs, grid_score = find_best_grid_trees(10)
print(f"Grid score: {grid_score:.6f}")

opt_xs, opt_ys, opt_degs, sa_score = sa_optimize(xs, ys, degs, TX, TY, max_iter=2000, seed=42)
print(f"SA score: {sa_score:.6f}")

ref_xs, ref_ys, ref_degs = fractional_refinement(opt_xs, opt_ys, opt_degs, TX, TY, max_iters=30)
final_score = score_group(ref_xs, ref_ys, ref_degs, TX, TY)
print(f"After refinement: {final_score:.6f}")

print(f"Has overlap: {has_any_overlap(ref_xs, ref_ys, ref_degs, TX, TY)}")

In [ ]:
# Generate all solutions with high-iteration SA
print("Generating solutions for N=1 to 200 with high-iteration SA...")
print("Iterations: N=1-20: 5000, N=21-50: 3000, N=51-100: 2000, N=101-200: 1000")

start_time = time.time()
solutions = {}  # n -> (xs, ys, degs)
scores = {}  # n -> score

for n in tqdm(range(1, 201)):
    if n == 1:
        # Optimal N=1 at 45 degrees
        xs = np.array([0.0], np.float64)
        ys = np.array([0.0], np.float64)
        degs = np.array([45.0], np.float64)
        scores[n] = score_group(xs, ys, degs, TX, TY)
        solutions[n] = (xs, ys, degs)
        continue
    
    # Get grid baseline
    xs, ys, degs, grid_score = find_best_grid_trees(n)
    
    # Determine iterations based on N
    if n <= 20:
        iterations = 5000
        num_restarts = 3
    elif n <= 50:
        iterations = 3000
        num_restarts = 2
    elif n <= 100:
        iterations = 2000
        num_restarts = 1
    else:
        iterations = 1000
        num_restarts = 1
    
    best_xs, best_ys, best_degs = xs.copy(), ys.copy(), degs.copy()
    best_score = grid_score
    
    # Multiple restarts
    for restart in range(num_restarts):
        seed = 42 + restart * 1000 + n
        opt_xs, opt_ys, opt_degs, sa_score = sa_optimize(xs, ys, degs, TX, TY, 
                                                          max_iter=iterations, seed=seed)
        
        # Fractional refinement
        ref_xs, ref_ys, ref_degs = fractional_refinement(opt_xs, opt_ys, opt_degs, TX, TY, max_iters=30)
        final_score = score_group(ref_xs, ref_ys, ref_degs, TX, TY)
        
        # Validate no overlap
        if not has_any_overlap(ref_xs, ref_ys, ref_degs, TX, TY) and final_score < best_score:
            best_score = final_score
            best_xs = ref_xs.copy()
            best_ys = ref_ys.copy()
            best_degs = ref_degs.copy()
    
    solutions[n] = (best_xs, best_ys, best_degs)
    scores[n] = best_score

elapsed = time.time() - start_time
print(f"\nTotal time: {elapsed/60:.1f} minutes")

In [ ]:
# Calculate total score
total_score = sum(scores[n] for n in range(1, 201))
print(f"Total score: {total_score:.6f}")
print(f"Target score: 68.947559")
print(f"Gap: {total_score - 68.947559:.6f}")

# Compare with previous best
prev_best = 87.364112  # From local_search
print(f"\nPrevious best: {prev_best:.6f}")
print(f"Improvement: {prev_best - total_score:.6f} points")

# Score breakdown
print("\nScore breakdown by N range:")
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores[n] for n in range(start, end + 1))
    print(f"  N={start:3d}-{end:3d}: {range_score:.4f} points")

In [ ]:
# Validate all configurations
print("Validating all configurations...")
overlap_issues = []

for n in range(1, 201):
    xs, ys, degs = solutions[n]
    if has_any_overlap(xs, ys, degs, TX, TY):
        overlap_issues.append(n)
        print(f"  N={n}: OVERLAP!")

if overlap_issues:
    print(f"\nWARNING: Overlaps found in {len(overlap_issues)} groups!")
else:
    print("\n\u2713 All configurations valid - no overlaps detected!")

In [ ]:
# Create submission
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n in range(1, 201):
    xs, ys, degs = solutions[n]
    assert len(xs) == n, f"Expected {n} trees, got {len(xs)}"
    for i_t in range(n):
        rows.append({
            "id": f"{n:03d}_{i_t}",
            "x": to_str(xs[i_t]),
            "y": to_str(ys[i_t]),
            "deg": to_str(degs[i_t]),
        })

submission = pd.DataFrame(rows)
print(f"Submission shape: {submission.shape}")
print(submission.head(10))

In [ ]:
# Save submission
import os
os.makedirs('/home/submission', exist_ok=True)
submission.to_csv('/home/submission/submission.csv', index=False)
submission.to_csv('/home/code/experiments/003_high_iter_sa/submission.csv', index=False)
print("Submission saved!")
print(f"\nFinal score: {total_score:.6f}")